<a target="_blank" href="https://colab.research.google.com/github/Tensor-Reloaded/Neural-Networks-Template-2025/blob/main/Lab02/Assignment1.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# **Assignment 1 (10 points)**

## **Solving a linear system in python**

In this homework, you will familiarize yourself with key linear algebra con-
cepts and Python programming by solving a system of linear equations. You
will explore multiple methods for solving such systems, including Cramer’s rule
and matrix inversion. By the end of this assignment, you will have a good un-
derstanding of how to represent and manipulate matrices and vectors in Python.

We begin with the following system of 3 linear equations with 3 unknowns:
$$ 2x + 3y - z = 5 $$
$$ x - y + 4z = 6 $$
$$ 3x + y + 2z = 7 $$

This system can be vectorized in the following form:
$$ A \cdot X = B $$
where:
$$
A = \begin{bmatrix}
2 & 3 & -1 \\
1 & -1 & 4 \\
3 & 1 & 2
\end{bmatrix}, \quad
X = \begin{bmatrix}
x \\
y \\
z
\end{bmatrix}, \quad
B = \begin{bmatrix}
5 \\
6 \\
7
\end{bmatrix}
$$

**Considerations**
- do not use any linear algebra framework such as $numpy$
- use python lists as data structures for matrices and vectors
- experiment with other values for the coefficients and free terms

### **1. Parsing the System of Equations (1 point)**

The first task is to implement a Python script that reads a system of linear equations from a text file and parses it into a matrix $A$ and a vector $B$. You will use the input format described below to extract the coefficients for $A$ and $B$.

**Input File Format**
```text
2x + 3y - z = 5
x - y + 4z = 6
3x + y + 2z = 7
```

Note that the coefficients are always in the order x, y and z and the terms are always space separated

In [34]:
import pathlib
import re
import math as m
import copy

def load_system(path: pathlib.Path) -> tuple[list[list[float]], list[float]]:
    file = open(path,"r")
    content = file.read().replace(" ","");

    values_matrix = []
    results_vector = []

    for line in content.split("\n"):
        left, right = line.split("=")
        left = left.replace("-","+-")
        row = []
        for member in left.split("+"):
            match = re.match(r'(-?\d*)([a-z])',member)
            if match:
                num, var = match.groups()
                length = len(num)
                if length == 1 and num[0] == '-':
                    num = -1
                elif length == 0:
                    num = 1
                else:
                    num = int(num)
            row.append(num)

        values_matrix.append(row)

        results_vector.append(right)
    file.close()
    return (values_matrix, results_vector)

A, B = load_system(pathlib.Path("system.txt"))
print(f"{A=} {B=}")

A=[[2, 3, -1], [1, -1, 4], [3, 1, 2]] B=['5', '6', '7']


### **2. Matrix and Vector Operations (5 points)**

Once you have successfully parsed the matrix and vector, complete the following exercises to manipulate and understand basic matrix and vector operations. Write Python functions for each of these tasks:

#### 2.1. Determinant

Write a function to compute the determinant of matrix $A$. Recall one of the formulae for the determinant of a $3x3$ matrix:
$$ \text{det}(A) = a_{11}(a_{22}a_{33} - a_{23}a_{32}) - a_{12}(a_{21}a_{33} - a_{23}a_{31}) + a_{13}(a_{21}a_{32} - a_{22}a_{31}) $$

In [45]:
def determinant(matrix: list[list[float]]) -> float:
    return float(matrix[0][0])*(float(matrix[1][1])*float(matrix[2][2])-float(matrix[1][2])*float(matrix[2][1])) - float(matrix[0][1])*(float(matrix[1][0])*float(matrix[2][2])-float(matrix[1][2])*float(matrix[2][0]))+float(matrix[0][2])*(float(matrix[1][0])*float(matrix[2][1])-float(matrix[1][1])*float(matrix[2][0]))

print(f"{determinant(A)=}")

determinant(A)=14.0


#### 2.2. Trace

Compute the sum of the elements along the main diagonal of matrix $A$. For a matrix $A$, this is:
$$ \text{Trace}(A) = a_{11} + a_{22} + a_{33} $$

In [37]:
def trace(matrix: list[list[float]]) -> float:
    return matrix[0][0] + matrix[1][1] + matrix[2][2]

print(f"{trace(A)=}")

trace(A)=3


#### 2.3. Vector norm

Compute the Euclidean norm of vector $B$, which is:
$$ ||B|| = \sqrt{b_1^2 + b_2^2 + b_3^2} $$

In [38]:
def norm(vector: list[float]) -> float:
    sum = 0
    for i in vector:
        i = int(i)
        sum+=i*i
    return m.sqrt(sum)

print(f"{norm(B)=}")

norm(B)=10.488088481701515


#### 2.4. Transpose of matrix

Write a function to compute the transpose of matrix $A$. The transpose of a matrix $A$ is obtained by swapping its rows and columns.
    

In [39]:
def transpose(matrix: list[list[float]]) -> list[list[float]]:
    m = len(matrix[0])
    n = len(matrix)
    transpose = [[0 for _ in range(n)] for _ in range(m)]
    for i in range(n):
        for j in range(m):
            transpose[j][i] = matrix[i][j]
    return transpose

print(f"{transpose(A)=}")

transpose(A)=[[2, 1, 3], [3, -1, 1], [-1, 4, 2]]


#### 2.5. Matrix-vector multiplication

Write a function that multiplies matrix $A$ with vector $B$.

In [43]:
def multiply(matrix: list[list[float]], vector: list[float]) -> list[float]:
    m = len(matrix[0])
    n = len(matrix)
    res = []
    for i in range(n):
        sum = 0
        for j in range(m):
            sum = sum + float(matrix[i][j]) * float(vector[j])
        res.append(sum)
    return res

print(f"{multiply(A, B)=}")

multiply(A, B)=[21.0, 27.0, 35.0]


### **3. Solving using Cramer's Rule (1 point)**

Now that you have explored basic matrix operations, solve the system of linear equations using Cramer's rule.

**Cramer's Rule:**

Cramer's rule allows you to solve for each unknown $x$, $y$, and $z$ using determinants. For example:
$$ x = \frac{\text{det}(A_x)}{\text{det}(A)}, \quad y = \frac{\text{det}(A_y)}{\text{det}(A)}, \quad z = \frac{\text{det}(A_z)}{\text{det}(A)} $$
where $A_x$, $A_y$, and $A_z$ are matrices formed by replacing the respective column of matrix $A$ with vector $B$.

In [46]:
def solve_cramer(matrix: list[list[float]], vector: list[float]) -> list[float]:
    m = len(matrix[0])
    n = len(matrix)
    bottom = determinant(matrix)
    if bottom == 0:
        print("!!!No possible solution for cramer, determinant is 0!!!")
    res = []
    for j in range(m):
        new_matrix = copy.deepcopy(matrix)
        print(new_matrix)
        for i in range(n):
            new_matrix[i][j] = vector[i]
        top = determinant(new_matrix)
        res.append(top/bottom)
    return res

print(f"{solve_cramer(A, B)=}")

[[2, 3, -1], [1, -1, 4], [3, 1, 2]]
[[2, 3, -1], [1, -1, 4], [3, 1, 2]]
[[2, 3, -1], [1, -1, 4], [3, 1, 2]]
solve_cramer(A, B)=[0.35714285714285715, 2.0714285714285716, 1.9285714285714286]


### **4. Solving using Inversion (3 points)**

Finally, solve the system by computing the inverse of matrix $A$ and multiplying it by vector $B$.
$$ A \cdot X = B \rightarrow X = A^{-1} \cdot B $$
**Adjugate Method for Matrix Inversion:**

To find the inverse of matrix $ A $, you can use the adjugate method:
$$ A^{-1} = \frac{1}{\text{det}(A)} \times \text{adj}(A) $$
where $\text{adj}(A)$ is the adjugate (or adjoint) matrix, which is the transpose of the cofactor matrix of $ A $.

**Cofactor Matrix:**

The cofactor matrix is a matrix where each element is replaced by its cofactor. The cofactor of an element $a_{ij}$ is given by:
$$ (-1)^{i+j} \times \text{det}(M_{ij}) $$
where $M_{ij}$ is the minor of element $a_{ij}$, which is the matrix obtained by removing the $i$-th row and $j$-th column from matrix $A$.

In [47]:
def minor(matrix: list[list[float]], i: int, j: int) -> list[list[float]]:
    return [row[:j] + row[j+1:] for r, row in enumerate(matrix) if r != i]

def det_2x2(matrix: list[list[float]]) -> float:
    return matrix[0][0]*matrix[1][1] - matrix[0][1]*matrix[1][0]

def cofactor(matrix: list[list[float]]) -> list[list[float]]:
    n = len(matrix)
    new = [[0 for _ in range(n)] for __ in range(n)]
    for i in range(n):
        for j in range(n):
            sign = 1 if (i + j) % 2 == 0 else -1
            new[i][j] = sign * det_2x2(minor(matrix, i, j))
    return new

def adjoint(matrix: list[list[float]]) -> list[list[float]]:
    return transpose(cofactor(matrix))

def inverse_of_matrix(matrix: list[list[float]]) -> list[list[float]]:
    det = determinant(matrix)
    adj = adjoint(matrix)
    return [[ val / det for val in row ] for row in adj ]

def solve(matrix: list[list[float]], vector: list[float]) -> list[float]:
   return multiply(inverse_of_matrix(matrix),vector)

print(f"{solve(A, B)=}")

solve(A, B)=[0.35714285714285765, 2.071428571428571, 1.9285714285714288]
